# 7. 데이터 준비하기: 다듬기, 변형, 병합
데이터 분석 시 실제로 데이터 준비하는데 시간 많이 필요 (약 80% 을 차지)

pandas 로 대부분 해결 가능

## 7.1 데이터 합치기


    pd.merge( on = 키이름 , how = 'outer' or 'inner' ): 하나 이상의 키를 따라 행을 이어 붙인다.
    pd.concat : 하나의 축을 따라 객체를 이어 붙인다.

## 7.1.1 데이터베이스 스타일로 DataFrame 합치기
merge, join : 관계형 데이터베이스의 핵심 연산

키(key)를 사용하여 로우(행)을 합친다.

### join의 개념 

    inner join : 교집합인 데이터들을 중심으로
    outer join : 합집합인 데이터들을 중심으로
    left join : 왼쪽 로우를 포함하는 결과
    right join : 오른쪽 로우를 포함하는 결과

pandas 의 pd.merge 함수 사용

pd.merge( ) 의 default : inner join


In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({'key': list('bbacaab') ,'key2': list('ddefgge'), 'data1' : range(7)})
df2 = pd.DataFrame({'key' : list('abc'),'key2' : list('ggf'), 'data2' : range(3)})

df1

,key,key2,data1
0,b,d,0
1,b,d,1
2,a,e,2
3,c,f,3
4,a,g,4
5,a,g,5
6,b,e,6


In [3]:
df2

,key,key2,data2
0,a,g,0
1,b,g,1
2,c,f,2


    pd.merge : 공통되는 열의 이름을 기준으로 그 열 안의 공통의 데이터를 추출하여 합친다.
            - 내부에 on 옵션을 걸어 어떤 열에 대해 병합할 것인지 설정할 수 있다.

옵션들
    1. on : 어떤 열을 key로 잡을 것인가
    2. how : 병합방식
        - inner
        - outer
        - left
        - right
    현재 df1,df2 의 공통되는 열은 'key'와 'key2'이며, 그 안의 공통 데이터는 'a','g'와 'c','f' 순서쌍 이다.

In [7]:
df1

,key,key2,data1
0,b,d,0
1,b,d,1
2,a,e,2
3,c,f,3
4,a,g,4
5,a,g,5
6,b,e,6


In [8]:
df2

,key,key2,data2
0,a,g,0
1,b,g,1
2,c,f,2


In [6]:
# 기본값으로 내부조인(inner join) 사용
# 현재 같은 열이름은  key, key2 이며, 그 안에서 겹치는 값은 (c,f),(a,g) 쌍이다.
## 이를 남기며 데이터 프레임 병합
pd.merge(df1,df2)

,key,key2,data1,data2
0,c,f,3,2
1,a,g,4,0
2,a,g,5,0


In [9]:
# 외부조인(outer join ) 사용 시 합집합 반환
# 공통되는 키(key, key2)에 대해 그 내부 요소의 합집합에 대해 데이터 병합
pd.merge(df1,df2 , how = 'outer') 

,key,key2,data1,data2
0,b,d,0.0,NaN
1,b,d,1.0,NaN
2,a,e,2.0,NaN
3,c,f,3.0,2.0
4,a,g,4.0,0.0
5,a,g,5.0,0.0
6,b,e,6.0,NaN
7,b,g,NaN,1.0


내부에 on 옵션을 걸어 어떤 열(key)에 대해 병합할 것인지 설정할 수 있다.

현재 겹치는 열이 'key' 와 'key2' 이지만 on 옵션을 통해 'key'로만 병합을 설정하였으므로 'key2'는 각각 데이터프레임의 열을 그대로 가져와서 붙인다.

In [10]:
# on = 'key' 로만 걸었기 때문에, 겹치는 열 이름인(key2) 는 각 데이터프레임 위치에 따라 _x, _y가 붙어서 반환
pd.merge(df1, df2, on = 'key')

,key,key2_x,data1,key2_y,data2
0,b,d,0,g,1
1,b,d,1,g,1
2,b,e,6,g,1
3,a,e,2,g,0
4,a,g,4,g,0
5,a,g,5,g,0
6,c,f,3,f,2


#### suffixes 옵션  : 병합된 컬럼에 대한 구분을 가능하도록 뒤에 문자를 덧붙여준다.

In [7]:
pd.merge(df1,df2, on = 'key', suffixes = ['_df1 것','_df2 것'])
# 겹치는 이름의 열이지만 join key 로 지정되지 못한 경우 그대로 테이블에 붙여지므로 구분이 필요

,data1,key,key2_df1 것,data2,key2_df2 것
0,0,b,d,1,g
1,1,b,d,1,g
2,6,b,e,1,g
3,2,a,e,0,g
4,4,a,g,0,g
5,5,a,g,0,g
6,3,c,f,2,f


#### 공통되는 key 가 없는 경우, 직접 지정하여 준다.

left_on, right_on 옵션

작동은 merge 와 같다. 즉 left_on과 right_on 에 대응되는 값이 key가 되어 병합

In [10]:
df3 = pd.DataFrame({'lkey' : list("bbacaab") , 'data1': range(7)})
df4 = pd.DataFrame({'rkey' : list("abd"), 'data2' : range(3)})

df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [11]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [20]:
df3

,rkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [21]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [22]:
pd.merge(df4, df3, how = 'left')

,rkey,data2,data1
0,a,0,2.0
1,a,0,4.0
2,a,0,5.0
3,b,1,0.0
4,b,1,1.0
5,b,1,6.0
6,d,2,NaN


In [12]:
# 두 데이터프레임에서 lkey와 rkey로 merge하고 싶은 경우, 두 key의 이름이 다르므로, on 옵션 대신
## left_on, right_on 사용
pd.merge(df3,df4, left_on = 'lkey', right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


### join의 개념 

    inner join : 교집합인 데이터들을 중심으로
    outer join : 합집합인 데이터들을 중심으로
    left join : 왼쪽 로우를 포함하는 결과
    right join : 오른쪽 로우를 포함하는 결과

In [13]:
df1

,key,key2,data1
0,b,d,0
1,b,d,1
2,a,e,2
3,c,f,3
4,a,g,4
5,a,g,5
6,b,e,6


In [14]:
df2

,key,key2,data2
0,a,g,0
1,b,g,1
2,c,f,2


In [12]:
pd.merge(df1,df2, how = 'left' )

,data1,key,key2,data2
0,0,b,d,NaN
1,1,b,d,NaN
2,2,a,e,NaN
3,3,c,f,2.0
4,4,a,g,0.0
5,5,a,g,0.0
6,6,b,e,NaN


In [15]:
pd.merge(df1,df2, how = 'right')

,key,key2,data1,data2
0,c,f,3.0,2
1,a,g,4.0,0
2,a,g,5.0,0
3,b,g,NaN,1


## 7.1.2 색인 머지하기

#### merge의 기준이 데이터의 색인(index or columns) 인 경우

left_index , right_index 옵션 사용(True or False)

In [23]:
left1 = pd.DataFrame({'key' : list('abaabc'), 'value' : range(6)})
right1 = pd.DataFrame({'group_val' : [3.5, 7]}, index = ['a','b'])

In [24]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [31]:
right1

,group_val
a,3.5
b,7.0


In [ ]:
merge(right_on = 열, right_index , left_index)

In [26]:
right1

Index(['a', 'b'], dtype='object')

In [19]:
pd.merge(left1, right1, left_on = 'key', right_index = True)
## left_on = 'key' : 병합 시 좌측에 들어가는 인자(left1)에서 'key'라는 열을 키로 사용한다.
## right_index : 병합 시 우측에 들어가는 인자(right1)의 색인을 키로 사용한다.

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


#### 데이터가 계층색인(중복 색인)인 경우
리스트로 여러 개의 열을 지정하여 머지해야 한다.

In [32]:
lefth = pd.DataFrame({'key1' : ['ohio','ohio','ohio','Nevada','Nevada'],
                      'key2' : [2001,2001,2002,2001,2002],
                      'data' : np.arange(5,)})

righth = pd.DataFrame(np.arange(12).reshape(6,2),
                     index = [['Nevada','Nevada', 'ohio','ohio','ohio','ohio'], [2001,2000,2000,2000,2001,2002]],
                     columns = ['event1', 'event2'])

In [33]:
lefth

,key1,key2,data
0,ohio,2001,0
1,ohio,2001,1
2,ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [36]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [35]:
righth.index

MultiIndex(levels=[['Nevada', 'ohio'], [2000, 2001, 2002]],
           codes=[[0, 0, 1, 1, 1, 1], [1, 0, 0, 0, 1, 2]])

In [23]:
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True)
# merge 수행 시 
# 좌측 데이터 프레임(lefth) 에서는 left_on 에 해당하는 열을 key 로 사용하고
# 우측 데이터 프레임(righth)에서는 인덱스를 key 로 사용하여 둘을 병합한다.

,key1,key2,data,event1,event2
0,ohio,2001,0,8,9
1,ohio,2001,1,8,9
2,ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [28]:
# 년도에 관련된 열로만 합치고 싶은 경우
right_re = righth.reset_index(level = 0)

pd.merge(lefth, right_re, left_on = 'key2', right_index = True)

,key1,key2,data,level_0,event1,event2
0,ohio,2001,0,Nevada,0,1
0,ohio,2001,0,ohio,8,9
1,ohio,2001,1,Nevada,0,1
1,ohio,2001,1,ohio,8,9
3,Nevada,2001,3,Nevada,0,1
3,Nevada,2001,3,ohio,8,9
2,ohio,2002,2,ohio,10,11
4,Nevada,2002,4,ohio,10,11


    dataframe객체.join()  
색인으로 머지할 때 사용, 열이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러개의 df 객체 병합 시 사용

In [29]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [30]:
right1

,group_val
a,3.5
b,7.0


In [31]:
left1.join(right1, how = 'outer')

,key,value,group_val
0,a,0.0,NaN
1,b,1.0,NaN
2,a,2.0,NaN
3,a,3.0,NaN
4,b,4.0,NaN
5,c,5.0,NaN
a,NaN,NaN,3.5
b,NaN,NaN,7.0


In [32]:
left1.join(right1, on = 'key') # left1 데이터의 'key'라는 열에 대해서 조인 수행

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## 7.1.3 축 따라 이어붙이기
    이어붙이기(concatenation) : numpy 에서 ndarray 를 이어붙이는 concatenate 함수 제공
    연결(binding)
    적층(stacking)

In [41]:
arr = np.arange(12).reshape(6,2)
arr

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11]])

In [45]:
np.concatenate([arr,arr,arr], axis = 1) # axis = 1 : 열을 따라 이어붙이기

array([[ 0,  1,  0,  1,  0,  1],
       [ 2,  3,  2,  3,  2,  3],
       [ 4,  5,  4,  5,  4,  5],
       [ 6,  7,  6,  7,  6,  7],
       [ 8,  9,  8,  9,  8,  9],
       [10, 11, 10, 11, 10, 11]])

In [46]:
np.concatenate([arr,arr], axis = 0) # axis = 0 : 행을 따라 이어붙이기

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11]])

    pd.concat() : 색인 대 색인으로 데이터프레임을 간단히 병합
    
            axis = 0 ( 행을 따라 이어붙이기 / 아래로 ) 가 default
            join = 'inner' : 겹치는 축에서 교집합에 대해 수행하려면 
            join_axes : 머지하려는 축을 직접 정해줄 수 있다.

In [47]:
s1 = pd.Series([0,1], index = ['a','b'])
s2 = pd.Series([2,3,4], index = ['c','d','e'])
s3 = pd.Series([5,6], index = ['f','g'])
s4 = pd.concat([s1 *5 , s3])

In [48]:
s1

a    0
b    1
dtype: int64

In [49]:
s2

c    2
d    3
e    4
dtype: int64

In [52]:
s3

f    5
g    6
dtype: int64

In [53]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [54]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [50]:
pd.concat([s1,s2,s3], axis = 1) # axis = 0 (열) 을 따라 이이붙이면 데이터프레임 형태가 된다.

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [56]:
pd.concat([s1,s2,s3], join = 'inner')

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [57]:
s1

a    0
b    1
dtype: int64

In [58]:
s4

a    0
b    5
f    5
g    6
dtype: int64

In [59]:
pd.concat([s1,s4], axis =1, join = 'inner') # 겹치는 색인에 대해(concat 과 inner join) 열 방향으로(가로로 axis = 1) 이어 붙였다.

,0,1
a,0,0
b,1,5


In [53]:
d1 = pd.DataFrame({'col1' : [100,200,300]}, index = ['a','b','c'])
d2 = pd.DataFrame({'col2' : [2,3,1]}, index = ['b','c','a'])

In [54]:
d1

,col1
a,100
b,200
c,300


In [55]:
d2

,col2
b,2
c,3
a,1


In [56]:
pd.concat([d1,d2], axis = 1)

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,col1,col2
a,100,1
b,200,2
c,300,3


join axes 예제

In [57]:
pd.concat([s1,s4], axis = 1, join_axes = [['a','c','b','e']])
# join_axes : 관심 있는 인덱스를 주어 그것에 대해서만 key로 삼아 join 수행한다.



,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


### keys옵션 
sereis 는 직접 확인하면서 각각 구분하기 힘들다.

따라서 계층 색인을 통해 하나의 series로 통합하여 보면 편하다.
 
    pd.concat( key = ) : 병합하려는 series 에게 각각 색인(이름)을 부여하여 계층적 색인을 입힌다.

In [58]:
s1

a    0
b    1
dtype: int64

In [59]:
s2

c    2
d    3
e    4
dtype: int64

In [60]:
s3

f    5
g    6
dtype: int64

In [61]:
pd.concat([s1,s2,s3], axis = 0)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [60]:
result = pd.concat([s1,s2,s3] , axis = 0, keys =['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [62]:
pd.concat([s1,s2,s3], axis = 1, keys = ['one','two','three'])
## axis = 1 로 주면 keys 리스트 안의 값들이 columns 의 이름이 된다.

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [63]:
daf1 = pd.DataFrame(np.arange(6).reshape(3,2), index = list('abc'), columns = 'one,two'.split(','))

daf2 = pd.DataFrame(np.arange(12,16).reshape(2,2), index = list('ac'), columns = 'three,four'.split(','))

In [64]:
daf1

,one,two
a,0,1
b,2,3
c,4,5


In [65]:
daf2

,three,four
a,12,13
c,14,15


In [65]:
pd.concat([daf1, daf2] , axis = 1, keys = ['level1','level2']) # 열(axis = 1)을 따라 이어 붙인다.
# daf1 에 해당하는 데이터는 level1 이라는 column이 붙어졌고
# daf2 에 해당하는 데이터는 level2 라는 column이 붙었다.

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2      
     one two  three  four
a      0   1   12.0  13.0
b      2   3    NaN   NaN
c      4   5   14.0  15.0

In [66]:
pd.concat({'level1' : daf1, 'level2' : daf2} , axis = 1) # 딕셔너리 형태로 넘겨 사용할 수도 있다.
# 딕셔너리의 키 값 : keys 옵션
# 딕셔너리의 value 값 : 각각 데이터프레임

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2      
     one two  three  four
a      0   1   12.0  13.0
b      2   3    NaN   NaN
c      4   5   14.0  15.0


    pd.concat( [  ,  ] , ignore_index = True )
        기존 색인 값을 '무시하고' 숫자 색인으로 된 초기화 색인 부여
각각 색인이 합칠 때 불필요한 경우

데이터 병합 후 중복 색인으로 문제가 되는 경우


In [74]:
# ignore_index 의 적절한 예
## 관찰 데이터가(record) 서로 다른 데이터프레임에 분리되어 있을 때
data1 = pd.DataFrame(np.random.randn(3,4), index = list('abc'))
data2 = pd.DataFrame(np.random.randn(2,4), index = list('ab'))

In [75]:
data1

,0,1,2,3
a,1.170124,0.367446,0.070259,-0.933616
b,-0.056854,1.030185,-0.925944,-0.617207
c,0.429032,-0.590994,-0.744486,-1.520624


In [76]:
data2

,0,1,2,3
a,0.624109,0.777386,0.159231,-1.443880
b,0.748472,0.190185,-0.430910,-0.106713


In [77]:
# 이렇게 하고 보면 'a' 와 'b' 색인이 겹쳐 나중에 다루기 어려워진다. 중복색인이기 때문
pd.concat([data1, data2])

,0,1,2,3
a,1.170124,0.367446,0.070259,-0.933616
b,-0.056854,1.030185,-0.925944,-0.617207
c,0.429032,-0.590994,-0.744486,-1.520624
a,0.624109,0.777386,0.159231,-1.443880
b,0.748472,0.190185,-0.430910,-0.106713


In [78]:
pd.concat([data1,data2], axis = 1) # 이렇게 할 수도 없는 노릇이고...

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2,3,0,1,2,3
a,1.170124,0.367446,0.070259,-0.933616,0.624109,0.777386,0.159231,-1.443880
b,-0.056854,1.030185,-0.925944,-0.617207,0.748472,0.190185,-0.430910,-0.106713
c,0.429032,-0.590994,-0.744486,-1.520624,NaN,NaN,NaN,NaN


In [79]:
pd.concat([data1,data2], ignore_index = True) # 이런식으로 하면 색인 데이터가 세로로 합쳐지며 새로운 색인이 깔끔하게 생성된다.

,0,1,2,3
0,1.170124,0.367446,0.070259,-0.933616
1,-0.056854,1.030185,-0.925944,-0.617207
2,0.429032,-0.590994,-0.744486,-1.520624
3,0.624109,0.777386,0.159231,-1.443880
4,0.748472,0.190185,-0.430910,-0.106713


## 7.1.4 겹치는 데이터 합치기

위의 merge , join 으로는 합치는 것이 불가능한 경우 : 색인의 일부나 전체가 겹치는 경우

    np.where : 벡터화 된 if-else 구문 사용


In [47]:
## 이거 좀 모르겠음..... p.265 ~ p.266

## 7.2 재형성과 피벗
표 형식의 데이터를 재배치

### 7.2.1 계층적 색인으로 재형성하기

    unstack : DataFrame 형태로(표 형태)
    stack : Series 형태로(세로로 길게) / 색인이 계층적

In [67]:
data = pd.DataFrame(np.arange(6).reshape(2,3),
                   index = pd.Index(['ohio','colorado'], name = 'state'),
                   columns = pd.Index(['one','two','three'], name = 'number'))
data

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [81]:
data_stk = data.stack() # 열에 있는 것을 행의 인덱스로 보낸다. columns -> rows
data_stk

state     number
ohio      one       0
          two       1
          three     2
colorado  one       3
          two       4
          three     5
dtype: int32

In [82]:
data_stk.unstack() # 행에 있는 것을 열의 columns 로 보낸다. rows -> columns / 기본적으로 가장 안쪽(오른쪽 인덱스)를 보낸다.

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [83]:
data_stk.unstack(0)
# 가장 바깥이 level 0 이다.

state,ohio,colorado
number,,
one,0,3
two,1,4
three,2,5


In [84]:
data_stk.unstack('state') # 위와 같이 레벨이나 순서번호를 지정하여 unstack 할 수 있다. 

state,ohio,colorado
number,,
one,0,3
two,1,4
three,2,5


## 7.2.2 피버팅으로 데이터 나열방식 바꾸기 

    dataframe객체.pivot(row로 보낼 열이름, columns으로 보낼 열 이름, dataframe에 채워 넣을 값)
        set_index 로 행에 대한 index 설정 -> unstack을 사용하여 형태를 변경하는 것을 한번에 해줌

        앞의 두 인자는 범주형 열이어야한다.

#### 다른 사용 예
    dataframe객체.pivot(' ' , ' '  )
        마지막 인자를 생략(지정해 주지 않음으로써)하여 계층적 색인(columns에)을 할 수 있다.(df에 column 값이 많은 경우)

In [68]:
import sklearn.datasets as sd
import pandas as pd

In [69]:
irisdb = sd.load_iris()

iris = pd.DataFrame(irisdb.data, columns = irisdb.feature_names)


irisdb.target


target_dict = dict(zip([0,1,2],irisdb.target_names))

target_data = [target_dict[tgdata] for tgdata in irisdb.target]

target_data

iris['species'] = target_data

In [70]:
iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [74]:
# 50개씩 잘라서 인덱스를 만들어준다.
iris['ind'] = [i % 50 for i in iris.index]

In [78]:
iris['ind']
# index 를 50개씩 잘라서 0 ~ 50, 0~ 50, 0~ 50 으로 만든다.

0       0
1       1
2       2
3       3
4       4
5       5
6       6
7       7
8       8
9       9
10     10
11     11
12     12
13     13
14     14
15     15
16     16
17     17
18     18
19     19
20     20
21     21
22     22
23     23
24     24
25     25
26     26
27     27
28     28
29     29
       ..
120    20
121    21
122    22
123    23
124    24
125    25
126    26
127    27
128    28
129    29
130    30
131    31
132    32
133    33
134    34
135    35
136    36
137    37
138    38
139    39
140    40
141    41
142    42
143    43
144    44
145    45
146    46
147    47
148    48
149    49
Name: ind, Length: 150, dtype: int64

In [98]:
iris.pivot('ind','species')
# 인자 순서 : row 에 들어갈 요소를 갖는 열이름, column에 들어갈 요소를 갖는 열이름

sepal length (cm)                      sepal width (cm)             \
species            setosa versicolor virginica           setosa versicolor   
ind                                                                          
0                     5.1        7.0       6.3              3.5        3.2   
1                     4.9        6.4       5.8              3.0        3.2   
2                     4.7        6.9       7.1              3.2        3.1   
3                     4.6        5.5       6.3              3.1        2.3   
4                     5.0        6.5       6.5              3.6        2.8   
5                     5.4        5.7       7.6              3.9        2.8   
6                     4.6        6.3       4.9              3.4        3.3   
7                     5.0        4.9       7.3              3.4        2.4   
8                     4.4        6.6       6.7              2.9        2.9   
9                     4.9        5.2       7.2              3.1        2.7   
10                    5.4        5.0       6.5              3.7        2.0   
11                    4.8        5.9       6.4              3.4        3.0   
12                    4.8        6.0       6.8              3.0        2.2   
13                    4.3        6.1       5.7              3.0        2.9   
14                    5.8        5.6       5.8              4.0        2.9   
15                    5.7        6.7       6.4              4.4        3.1   
16                    5.4        5.6       6.5              3.9        3.0   
17                    5.1        5.8       7.7              3.5        2.7   
18                    5.7        6.2       7.7              3.8        2.2   
19                    5.1        5.6       6.0              3.8        2.5   
20                    5.4        5.9       6.9              3.4        3.2   
21                    5.1        6.1       5.6              3.7        2.8   
22                    4.6        6.3       7.7              3.6        2.5   
23                    5.1        6.1       6.3              3.3        2.8   
24                    4.8        6.4       6.7              3.4        2.9   
25                    5.0        6.6       7.2              3.0        3.0   
26                    5.0        6.8       6.2              3.4        2.8   
27                    5.2        6.7       6.1              3.5        3.0   
28                    5.2        6.0       6.4              3.4        2.9   
29                    4.7        5.7       7.2              3.2        2.6   
30                    4.8        5.5       7.4              3.1        2.4   
31                    5.4        5.5       7.9              3.4        2.4   
32                    5.2        5.8       6.4              4.1        2.7   
33                    5.5        6.0       6.3              4.2        2.7   
34                    4.9        5.4       6.1              3.1        3.0   
35                    5.0        6.0       7.7              3.2        3.4   
36                    5.5        6.7       6.3              3.5        3.1   
37                    4.9        6.3       6.4              3.6        2.3   
38                    4.4        5.6       6.0              3.0        3.0   
39                    5.1        5.5       6.9              3.4        2.5   
40                    5.0        5.5       6.7              3.5        2.6   
41                    4.5        6.1       6.9              2.3        3.0   
42                    4.4        5.8       5.8              3.2        2.6   
43                    5.0        5.0       6.8              3.5        2.3   
44                    5.1        5.6       6.7              3.8        2.7   
45                    4.8        5.7       6.7              3.0        3.0   
46                    5.1        5.7       6.3              3.8        2.9   
47                    4.6        6.2       6.5              3.2        2.9   
48                    5.3     

In [79]:
import multiprocessing

In [82]:
multiprocessing.cpu_count()

8

In [83]:
import multiprocessing as mp

In [88]:
a = [i for i in range(10)]

In [89]:
a1 = a[:5]
a2 = a[5:]

In [86]:
p = mp.Pool( processes = 2)

In [87]:
def multiple2(x):
    return x * 2

In [ ]:
p.map(multiple2, [a1,a2])

In [ ]:
parallel python

## 7.3 데이터 변형


### 7.3.1 중복 제거하기

    dataframe객체.duplicated( ) : 각 row 가 중복인지 불리언 타입으로 반환
    dataframe객체.drop_duplicates( ) : 겹치는 데이터를 하나만 살리고 나머지 삭제
    dataframe객체.drop_duplicates( "열" ) : 특정 열에 기반하여 중복 삭제

default : 중복값들 중 처음 발견된 값(가장 위에 있는 값)을 False 로 놓고 삭제되지 않게 한다.

keep = 'last' 옵션 : 가장 마지막 중복 값을 삭제되지 않게 한다.


In [1]:
import pandas as pd

In [2]:
dudata = pd.DataFrame({'key1' : ['one'] * 4 + ['two'] * 4 , 
                       'key2' : [1,1,1,2,3,3,4,4]}) 
dudata

,key1,key2
0,one,1
1,one,1
2,one,1
3,one,2
4,two,3
5,two,3
6,two,4
7,two,4


In [3]:
dudata.duplicated() # 겹치는 것을 True로 반환 , 겹치는 것이 3개 이면 2개만 True로 해서 하나는 False로 살려둔다.

0    False
1     True
2     True
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [4]:
dudata.drop_duplicates() # unique 한 데이터로 구성된 데이터 프레임을 생성해준다.

,key1,key2
0,one,1
3,one,2
4,two,3
6,two,4


In [3]:
dudata['v1'] = range(8)
dudata

,key1,key2,v1
0,one,1,0
1,one,1,1
2,one,1,2
3,one,2,3
4,two,3,4
5,two,3,5
6,two,4,6
7,two,4,7


In [4]:
# 모든 row의 값들이 각각 다르므로 지워지는 row가 없다.
dudata.drop_duplicates()

,key1,key2,v1
0,one,1,0
1,one,1,1
2,one,1,2
3,one,2,3
4,two,3,4
5,two,3,5
6,two,4,6
7,two,4,7


In [6]:
# key2열에서 겹치는 row를 추려 삭제한다.
# keep = 'last' 옵션을 통해 겹치는값중 가장 밑에 있는 값을 살린다.
dudata.drop_duplicates(['key1','key2'], keep = 'first') 


,key1,key2,v1
0,one,1,0
3,one,2,3
4,two,3,4
6,two,4,6


## 7.3.2 함수나 매핑 이용해 데이터 변형하기
    map 메서드 : 한 객체(주로 Series/ 하나의 열) 안에서 값의 부분집합을 변경하는데 사용
    
    Series객체.map(값을 변경할 딕셔너리, 함수 등등)

In [7]:
dudata

,key1,key2,v1
0,one,1,0
1,one,1,1
2,one,1,2
3,one,2,3
4,two,3,4
5,two,3,5
6,two,4,6
7,two,4,7


In [20]:
dudata['key2']

0    1
1    1
2    1
3    2
4    3
5    3
6    4
7    4
Name: key2, dtype: int64

In [8]:
dudata['key2'].map({1 : 100, 2 : 200})

0    100.0
1    100.0
2    100.0
3    200.0
4      NaN
5      NaN
6      NaN
7      NaN
Name: key2, dtype: float64

In [18]:
dudata['key1'].map('I am {}'.format)

0    I am one
1    I am one
2    I am one
3    I am one
4    I am two
5    I am two
6    I am two
7    I am two
Name: key1, dtype: object

In [18]:
dudata['key2'].map(lambda x : x if x < 4 else 1000)

0       1
1       1
2       1
3       2
4       3
5       3
6    1000
7    1000
Name: key2, dtype: int64

## 7.3.3 값 치환하기
    
    dataframe객체.replace(바꾸고 싶은 값, 바꿀 값) : 바꿀 값이 여러 개인 경우 인자에 리스트를 넘긴다.


주로 df.replace(바꾸고 싶은 값, np.nan) 으로 Na 로 치환 후 df.fillna( ) 를 사용하여 치환

In [22]:
ovdata = pd.DataFrame({'a' : [1,1,2,3,4,5,6,6,7] , 'b' : [3,4,4,5,2,1,4,6,3]} )
ovdata

,a,b
0,1,3
1,1,4
2,2,4
3,3,5
4,4,2
5,5,1
6,6,4
7,6,6
8,7,3


In [24]:
ovdata.replace([1,6],['1 치환','6 치환']) # 각각 변경하려는 값을 짝을 지어 변경할 수 있다.

,a,b
0,1 치환,3
1,1 치환,4
2,2,4
3,3,5
4,4,2
5,5,1 치환
6,6 치환,4
7,6 치환,6 치환
8,7,3


In [21]:
ovdata.replace({1:'사전으로 1 치환', 6 : '사전으로 6 치환'}) # 딕셔너리 형태로 짝을 지어 치환가능

,a,b
0,사전으로 1 치환,3
1,사전으로 1 치환,4
2,2,4
3,3,5
4,4,2
5,5,사전으로 1 치환
6,사전으로 6 치환,4
7,사전으로 6 치환,사전으로 6 치환
8,7,3


## 7.3.4 축 색인 이름 바꾸기

    df.rename ( index =   , columns =   , inplace = )
    row 나 column 의 이름을 변경할 때 사용

- 원본 데이터를 바로 변경하는 경우 _ = df.rename( index = , columns = , inplace = True)

- 일부만 변경하는 경우 df.rename( index = {변경 하려는 인덱스 : 변경 하려는 인덱스 이름} , columns = )

In [29]:
data = pd.DataFrame(np.random.randint(0,10,size = (3,4)),
                   index = list('abc'),
                   columns = list('ABCD'))

In [36]:
data

,A,B,C,D
a,2,5,6,6
b,7,4,5,8
c,3,8,5,4


In [39]:
# 원래 값 : 바꾸려는 값 의 형식으로 딕셔너리를 넘겨주어 열이나 행이름 변경
# inplace= True 가 아닌 경우 바뀐 데이터 프레임을 보여줄 뿐이다.
data.rename(columns = {'A' : 'A_re', 'B' : 'B_re'},
           index = {'c' : 'c_re'})

,A_re,B_re,C,D
a,2,5,6,6
b,7,4,5,8
c_re,3,8,5,4


In [40]:
data

,A,B,C,D
a,2,5,6,6
b,7,4,5,8
c,3,8,5,4


In [41]:
# 바꾼후 객체가 바뀌기 원하는 경우 inplace = True 옵션 필요
data.rename(columns = {'A': 'A_re'}, inplace = True)

data

,A_re,B,C,D
a,2,5,6,6
b,7,4,5,8
c,3,8,5,4


## 7.3.5 개별화와 양자화

분류나 그룹화가 필요한 연속형 데이터를 다룰 때 적합

    pd.cut(데이터, 분할 기준 , right = T or F , labels = [분할 구간마다 부여할 이름] )

In [30]:
ages = np.random.randint(40, size = 20)

In [31]:
ages

array([24, 29, 10,  7, 10, 17, 26, 22, 34, 31,  9, 32,  2,  0, 27, 11, 13,
        6, 30, 38])

In [36]:
classify = np.arange(0,50,10)
classify # 나이를 10씩 나눈다.

array([ 0, 10, 20, 30, 40])

In [37]:
cats = pd.cut(ages, classify)  # age 데이터를 classify 구간에 맞추어 자른다.

# 각 데이터가 속한 구간에 대한 정보가 나옵니다.
cats

[(20, 30], (20, 30], (0, 10], (0, 10], (0, 10], ..., (10, 20], (10, 20], (0, 10], (20, 30], (30, 40]]
Length: 20
Categories (4, interval[int64]): [(0, 10] < (10, 20] < (20, 30] < (30, 40]]

    Categories 라는 특수한 객체 : 그룹이름이 담긴 배열 / 그룹 구간

In [41]:
ages.min()

0

In [42]:
ages.max()

38

In [44]:
cats.codes

array([ 2,  2,  0,  0,  0,  1,  2,  2,  3,  3,  0,  3,  0, -1,  2,  1,  1,
        0,  2,  3], dtype=int8)

In [45]:
cats.categories

IntervalIndex([(0, 10], (10, 20], (20, 30], (30, 40]],
              closed='right',
              dtype='interval[int64]')

   
       pd.cut(데이터, 분류 기준 right = ) 
           right 인자 : 분류 구간의 포함(포함 괄호의 위치)을( '(' 와 '[' ) 조절할 수 있다.

In [51]:
# right = False : 각 구간의 오른쪽에 ']' 를 사용하지 않고 ')' 를 사용하겠다.
cats2 = pd.cut(ages, classify , right = False) 

In [52]:
cats2.categories

IntervalIndex([[0, 10), [10, 20), [20, 30), [30, 40)],
              closed='left',
              dtype='interval[int64]')

In [54]:
ages

array([ 7, 27,  3, 14,  7, 27,  9, 38, 31, 27, 15, 26, 37, 13, 14,  2, 19,
       27, 13, 27])

In [55]:
# 각 개체가 속한 class에 대한 정보
cats2.codes

array([0, 2, 0, 1, 0, 2, 0, 3, 3, 2, 1, 2, 3, 1, 1, 0, 1, 2, 1, 2],
      dtype=int8)

    pd.cut(데이터, 분류기준, labels = )
        label인자 : 그룹의 이름을 지정해줄 수 있다.

In [47]:
cat_label_result =pd.cut(ages, classify , labels = ['youth','youngadult','middleaged','senior'])

In [48]:
cat_label_result.categories

Index(['youth', 'youngadult', 'middleaged', 'senior'], dtype='object')

In [49]:
cat_label_result.codes

array([ 2,  2,  0,  0,  0,  1,  2,  2,  3,  3,  0,  3,  0, -1,  2,  1,  1,
        0,  2,  3], dtype=int8)

In [50]:
cat_label_result.categories[cat_label_result.codes]

Index(['middleaged', 'middleaged', 'youth', 'youth', 'youth', 'youngadult',
       'middleaged', 'middleaged', 'senior', 'senior', 'youth', 'senior',
       'youth', 'senior', 'middleaged', 'youngadult', 'youngadult', 'youth',
       'middleaged', 'senior'],
      dtype='object')

    pd.qcut( ) : 변위치를 기준으로 그룹마다 동일한 크기로 알아서 분류하여준다.
    pd.cut( ) : 분류 구간을 정해주기 때문에 그룹마다 크기가 다르게 나온다.

#### 비교 pd.cut( ) VS pd.qcut( )
- pd.cut( ) : 사용자가 지정한 구간 / 구간마다 들어간 데이터 크기가 다를 수 있다.
- pd.qcut( ) : 해당 데이터 집단의 quantile 로 구간을 정함 / 구간마다 데이터 크기가 비슷

In [51]:
randata = np.random.randn(500)

In [52]:
randata

array([ 8.88481993e-01,  7.75370302e-01,  8.77299200e-01,  1.78926602e+00,
        8.19572042e-01,  5.07915870e-01,  5.08488653e-01,  4.83706785e-01,
        5.02529041e-02, -5.72140885e-01, -2.95108862e-01,  2.41229044e-01,
        7.72667299e-01, -1.15534583e-03, -1.67622095e-01, -8.30007034e-02,
       -9.58151282e-02,  6.85488891e-01, -9.73909124e-01,  1.26022236e+00,
        3.56065414e-01, -1.23802104e+00,  3.62588147e-01, -1.42748274e+00,
       -1.07758547e+00,  4.51261624e-01, -3.57239749e-01,  1.29832926e+00,
        6.30177913e-01,  3.02119239e-02, -7.53887083e-01, -1.47828164e+00,
       -1.35302969e+00, -1.29708945e-02, -1.18418503e+00, -5.16168454e-01,
       -4.55910626e-01, -1.96527913e+00, -1.03480128e+00, -1.23719347e+00,
       -3.28179367e-01, -1.14552908e-01,  1.00240071e+00,  4.26229991e-01,
       -2.01863207e+00, -1.16892108e+00, -1.33226777e+00, -4.51560073e-01,
        7.04304043e-02, -3.28239690e-01,  1.08020055e+00, -1.16162602e+00,
        1.98167887e-01, -

In [53]:
pd.qcut(randata, 4).categories # 4개의 구간으로 나누어라 / 즉 각각 quantile 을 기준으로 나누어라

IntervalIndex([(-3.602, -0.718], (-0.718, -0.0572], (-0.0572, 0.657], (0.657, 2.746]],
              closed='right',
              dtype='interval[float64]')

In [60]:
pd.value_counts(pd.qcut(randata,4)) # value_counts 로 각각의 구간에 속해있는 데이터의 값을 센다.
# 구간마다 데이터 크기가 동일한 것을 확인할 수 있다.

(0.625, 3.933]      125
(0.0205, 0.625]     125
(-0.681, 0.0205]    125
(-3.335, -0.681]    125
dtype: int64

In [61]:
pd.qcut(randata, np.arange(0,1,0.1)) # 변위치를 직접 지정해 줄 수 있다.

[(0.822, 1.339], (-0.291, 0.0205], (-0.54, -0.291], (0.492, 0.822], (-3.335, -1.255], ..., (-1.255, -0.835], (0.0205, 0.248], (-3.335, -1.255], (-0.835, -0.54], (0.492, 0.822]]
Length: 500
Categories (9, interval[float64]): [(-3.335, -1.255] < (-1.255, -0.835] < (-0.835, -0.54] < (-0.54, -0.291] ... (0.0205, 0.248] < (0.248, 0.492] < (0.492, 0.822] < (0.822, 1.339]]

## 7.3.6 특이값 찾아내고 제외하기
### 

In [62]:
np.random.seed(12345)

In [63]:
dataf = pd.DataFrame(np.random.randn(1000,4)) # 1000 by 4 사이즈인 랜덤 값을 갖는 ndarray를 생성한다.

In [64]:
dataf

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [108]:
dataf[3][np.abs(dataf[3]) > 3] # 절댓값이 3을 초과하는 이름이 3인 열의 값들

365   -3.555502
664    3.493070
895   -3.103480
Name: 3, dtype: float64

    조건.any(축) : 행(0)/ 열(1) 에 대해 해당 조건을 만족하는 것이 하나라도 있으면 해당 축(행 or 열) 전체를 반환한다.

In [67]:
dataf.columns = ['a','b','c','d']

In [68]:
dataf

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [69]:
(np.abs(dataf) > 3)

,a,b,c,d
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
5,False,False,True,False
6,False,False,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [70]:
# 각 열에 대해 해당 조건이 만족하는지
(np.abs(dataf) > 3).any(0)

a    True
b    True
c    True
d    True
dtype: bool

In [72]:
# 각 행에 대해 해당 조건이 만족하는지
(np.abs(dataf) > 3).any(1)

0      False
1      False
2      False
3      False
4      False
5       True
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
970    False
971    False
972    False
973    False
974    False
975    False
976    False
977    False
978    False
979    False
980    False
981    False
982    False
983    False
984    False
985    False
986    False
987    False
988    False
989    False
990    False
991    False
992    False
993    False
994    False
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [159]:
dataf[(np.abs(dataf) > 3).any(1)] # 열에 대해서(axis = 1) 메서드를 수행하고 행들을 반환한다(?)
# 즉, 3을 초과하는 데이터가 하나라도 있으면 해당 행을 반환

,0,1,2,3
17,-0.843675,-0.657652,3.107432,-1.285660
26,-1.613277,-1.011946,-3.184964,3.262894
60,-0.241460,-1.697015,0.479941,-3.206012
230,0.650475,0.460878,-3.440306,-0.347412
251,-1.038172,-0.098929,-0.171646,3.423630
280,1.125926,-3.306159,-0.479018,0.441948
287,3.328181,0.606253,-0.731191,-0.037887
383,0.829159,3.541876,0.285995,0.709384
652,-1.595629,-0.632807,0.229110,3.183761
746,1.599890,3.142646,-0.046947,1.839707


    np.sign(dataframe객체) : 음수는 -1 로 양수는 +1 로 바꾸어준다.

In [73]:
np.sign(dataf)

,a,b,c,d
0,-1.0,1.0,-1.0,-1.0
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,1.0,1.0
4,-1.0,-1.0,1.0,-1.0
5,-1.0,1.0,1.0,-1.0
6,-1.0,1.0,1.0,1.0
7,1.0,1.0,-1.0,-1.0
8,-1.0,-1.0,-1.0,1.0
9,-1.0,1.0,-1.0,1.0


In [74]:
dataf[np.abs(dataf) > 3] = np.sign(dataf) * 3
# 절댓값이 3을 초과하는 값들을 추출한 다음
# +1 이나 -1 로 변환하고 3을 곱해 넣어준다.
# 즉 최댓값을 3 / 최솟값을 -3으로 변환하여주는 것이다.

In [75]:
dataf.describe()

,a,b,c,d
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


## 7.3.7 치환과 임의 샘플링
    np.random.permutation( 바꾸고 싶은 만큼을 길이 )

In [76]:
pdata = pd.DataFrame(np.arange(20*4).reshape(20,4))
pdata

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23
6,24,25,26,27
7,28,29,30,31
8,32,33,34,35
9,36,37,38,39


In [77]:
# 0 ~ 3까지의 값을 랜덤으로 순서를 매겨 반환
sampler = np.random.permutation(4)
sampler

array([1, 3, 2, 0])

In [78]:
# 추출한 값에 대응하는 row 를 가져온다.
pdata.take(sampler)

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11
0,0,1,2,3


In [79]:
# iloc 를 사용하여도 된다.
pdata.iloc[sampler, :]

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
2,8,9,10,11
0,0,1,2,3


#### '임의로' 뽑아서 '일부분만' 재배치 하고 싶은 경우
    np.random.permutation(len(df)) 로 행 전체에 대해 랜덤 순서를 매긴다.
    np.random.permutation(len(df))[: 10] : 랜덤 순서에서 앞의 10개만 뽑는다

즉 결과는 전체에서 랜덤하게 10개를 뽑아 배치시키는 것!!!

In [80]:
pdata.take(np.random.permutation(len(pdata))[:10])

,0,1,2,3
7,28,29,30,31
9,36,37,38,39
11,44,45,46,47
5,20,21,22,23
1,4,5,6,7
15,60,61,62,63
0,0,1,2,3
17,68,69,70,71
3,12,13,14,15
8,32,33,34,35


In [81]:
# np.random.permutation(리스트) 도 가능
np.random.permutation(['a','b',100,200])
# 다 문자로 취급하는구만



array(['a', '100', 'b', '200'], dtype='<U3')

## 7.3.8 표시자/ 더미 변수

더미 변수란(dummies, one-hot-encoding)? 
- 데이터가 범주형인 경우 : 해당 열에 k 가지의 범주형 값이 있을 때 k개의 열이 있는 df를 만들고 0과 1로 채우는 것
- 통계모델(0,1코딩/ 팩토리얼 디자인) 이나 기계학습 앱에 주로 사용

        
        
     pd.get_dummies(원하는 열- 범주형) 함수 
     
     
범주형 변수에 대해서는 해당 열의 type을 정확히 지정한 수 실행해야한다.

numeric 열에 대해서는 실행되지 않고 그대로 반환된다.

In [54]:
df = pd.DataFrame({'key' : list('bbacab'),
                  'data1' : range(6,12)})
df

,key,data1
0,b,6
1,b,7
2,a,8
3,c,9
4,a,10
5,b,11


In [85]:
# key 는 object type 으로 범주형
# data1 은 numeric type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
key      6 non-null object
data1    6 non-null int64
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


In [86]:
pd.get_dummies(df)
# key 열이 범주형 데이터를 취급한다. ( 'a' 'b' 'c' )
# 데이터의 종류가 a,b,c 로 3개 이므로 열을 3개 만들고 0,1 코딩을 한다.
# 현재 data1 은 numerical 로 범주형 데이터가 아니므로 dummies 가 실행되지 않는다.
# 즉 문자열 데이터에 대해서 수행한다.

# key 열에 범주형 데이터 a,b,c (3개) 가 있으므로 해당 이름인 열 key_a, key_b, key_c 를 만들고
# 해당 행에 있으면 1 , 없으면 0 으로 코딩한다.

,data1,key_a,key_b,key_c
0,6,0,1,0
1,7,0,1,0
2,8,1,0,0
3,9,0,0,1
4,10,1,0,0
5,11,0,1,0


In [87]:
k = pd.DataFrame({'key' : list('1234455')})
# 숫자를 문자열로 넣어 범주형으로 취급하면 아래와 같이 dummies 가 실행된다.

In [89]:
#보기에는 숫자이지만 type 은 string(object)이다
k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 1 columns):
key    7 non-null object
dtypes: object(1)
memory usage: 136.0+ bytes


In [88]:
pd.get_dummies(k['key'])

,1,2,3,4,5
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0
5,0,0,0,0,1
6,0,0,0,0,1


In [101]:
pd.get_dummies(df['key']) # 열 하나만 지정하여 0,1 코딩을 할 수 있다.

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


#### 여러 범주형 열을 넣는 경우

In [61]:
many_category = pd.DataFrame({'col1' : ['a','b','c','c','b','a','a','b'],
                                           'col2' : ['A','B','B','B','C','A','A','B'] })

In [62]:
many_category

,col1,col2
0,a,A
1,b,B
2,c,B
3,c,B
4,b,C
5,a,A
6,a,A
7,b,B


In [63]:
pd.get_dummies(many_category)
# 이렇게 많은 열에 대해서도 dummy를 만들 수 있다.

,col1_a,col1_b,col1_c,col2_A,col2_B,col2_C
0,1,0,0,1,0,0
1,0,1,0,0,1,0
2,0,0,1,0,1,0
3,0,0,1,0,1,0
4,0,1,0,0,0,1
5,1,0,0,1,0,0
6,1,0,0,1,0,0
7,0,1,0,0,1,0


#### get_dummies 와 cut 같이 사용하기 
통계 애플리케이션에서 유용하게 사용 가능

In [64]:
# 전체 데이터
values = np.random.randint(10, size = 10)

In [65]:
values

array([1, 3, 8, 3, 2, 3, 9, 8, 1, 1])

In [66]:
# 나눌 구간 정하기
bins = [i for i in range(0,11,2)] # 0,2,4,6...,10 으로 구간을 나눈다.
bins

[0, 2, 4, 6, 8, 10]

In [67]:
pd.get_dummies(pd.cut(values,bins))
# cut 한 객체를 get_dummies 시키면 , 각 구간에 따라 0,1 coding 이 된다.
# 해당 구간에 있는 경우 1
# 없는 경우 0

   (0, 2]  (2, 4]  (4, 6]  (6, 8]  (8, 10]
0       1       0       0       0        0
1       0       1       0       0        0
2       0       0       0       1        0
3       0       1       0       0        0
4       1       0       0       0        0
5       0       1       0       0        0
6       0       0       0       0        1
7       0       0       0       1        0
8       1       0       0       0        0
9       1       0       0       0        0

## 7.4 문자열 다루기
파이썬의 장점 중 하나 : 문자열 처리 용이


기본적으로 내장함수로 많은 처리가 가능하나 pandas에서 제공하는 메서드로도 많이 사용

### 7.4.1 문자열 객체 메서드
    split( )
    strip( ) : 공백문자를 제거
    "  ".join(리스트) : 리스트 안의 문자열 요소끼리 사이를 " "으로 채워 연결하여 문자열 반환

In [68]:
a = 'python is so easy'
a.split(' ')

['python', 'is', 'so', 'easy']

#### 내부에서 문자열 찾아 인덱스 반환하기
    문자열.index('찾으려는 문자'): 문자열의 인덱스 반환 / 없는 경우 error 발생
    문자열.find('찾으려는 문자') : 문자열의 인덱스 반환/ 없는 경우 -1 반환


찾으려는 문자가 여러번 나오는 경우 가장 앞의 인덱스만 반환한다.

In [74]:
a.index('o')

4

In [75]:
# err
a.find('o')

4

In [109]:
#a.idex('없는 문자') - 없는 문자를 찾으려니 에러발생

In [110]:
a.find('없는 문자') # 문자가 해당 문자열에 없는 경우 -1 반환

-1

In [111]:
# 문자열.find() 와 문자열.index() 의 차이
# find 는 에러가 발생하지 않는다

In [112]:
split_a = a.split(' ')
split_a

['python', 'is', 'so', 'easy']

In [113]:
adot = '::'.join(split_a) # 리스트 요소 값 사이사이에 :: 를 넣고 붙여 한 문자열로 만든다.
adot

'python::is::so::easy'

In [114]:
a.count('o') # 해당 문자열의 수를 센다.

2

In [115]:
adot.replace('::','$$')

'python$$is$$so$$easy'

In [116]:
a[:4].upper() + a[4:]

'PYTHon is so easy'

In [117]:
a.lower()

'python is so easy'

### 7.4.2 정규표현식
문자열의 '패턴' 을 찾는 유연한 방법

패턴(정규표현식) 을 지정하여 텍스트를 걸러내는 작업

regex : 단일표현식/ 정규표현언어로 구성된 문자열
 
#### re 모듈함수 
1. 패턴 매칭
2. 치환 분리 : regex.sub('대체할 텍스트', 가공할 문자열) - 가공할 문자열에서 regex(패턴)에 해당하는 텍스트를   '대체할 텍스트'로 치환
3. 분리

#### 정규표현식 메서드

    정규표현식 객체 이름(regex) = re.compile(패턴 문자열 , flags =    )
    regex.split(가공할 문자열) : 패턴과 일치하는 것으로 분리
    regex.findall(가공할 문자열) : 패턴과 일치하는 것 모두 찾기
    regex.search(가공할 문자열) : 패턴과 일치하는 첫번째 요소 찾기
    regex.matc(가공할 문자열) : 문자 처음부분부터  찾는다. 만약 첫 부분부터 없다면 None 반환
    regex.sub(' 대체할 문자열 ' , 가공할 문자열 ) : 패턴에 일치하는 것들을 대체할 문자열로 모두 치환

In [98]:
import re

In [99]:
texts = 'foo    bar\t baz   \tqux'
print(texts)

foo    bar	 baz   	qux


In [100]:
re.split('\s+',texts) # re.split(정규표현식, 적용하려는 문자열)  // 여기서 '\s+' 는 하나 이상 띄어져 있는 문자열이라는 뜻
# re.split : 정규표현식을 사용하여 문자를 추출하는 유연한 split 함수/ 내장 split 함수보다 유연
# 작동 순서
#1. 정규표현식이 먼저 컴파일 된다.
#2. split 메서드가 실행

['foo', 'bar', 'baz', 'qux']

#### 정규표현식을 객체로 저장하여 사용 가능

In [101]:
regex = re.compile('\s+')
# '\s+'(하나 이상 띄어져 있는 문자) 라는 정규표현식을 compile 하여 객체로 저장한다.

In [102]:
regex.split(texts)
# 정규표현식 객체.split( 문자열 ) : 문자열에 정규표현식을 적용하여 split 한다.

['foo', 'bar', 'baz', 'qux']

In [103]:
regex.findall(texts) # 정규 표현식에 매칭되는 텍스트 값을 얻기

['    ', '\t ', '   \t']

In [104]:
regex.search(texts) # 문자열에서 패턴과 일치하는 첫번째 존재 반환
# 즉 findall 에서 나온 리스트의 가장 첫번째 요소값이 반환되는 것임 ㅇㅇ

<_sre.SRE_Match object; span=(3, 7), match='    '>

In [105]:
txt = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gamil.com"""

In [106]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # 이메일 텍스트에서 각각 요소를 분리할 패턴(정규표현식)을 지정해준다.,

## 여기서 패턴은 sample@saple.com 등과 같은 이메일 형식이다.

In [107]:
regex2 = re.compile(pattern, flags = re.IGNORECASE) # 정규 표현식에 여러 옵션을 걸어 하나의 표현식 객체로 컴파일 한다.
# 여기서 옵션 flags = IGNORECASE 는 '대소문자 구분없이' 라는 뜻이다.

In [108]:
regex2.split(txt) # 정규표현 패턴(여기서는 이메일 부분) 을 기준으로 나눈다.

['Dave ', '\nSteve ', '\nRob ', '']

In [109]:
regex2.findall(txt) # regex2라는 정규 표현식 패턴을 txt 라는 문자열에 적용하여 작동한다.

# 정규표현식을 만족하는(매치되는) 문자열을 반환한다.

['dave@google.com', 'steve@gmail.com', 'rob@gamil.com']

In [110]:
regex2.search(txt)
# 정규표현식에 매칭되는 가장 첫번째 값
# findall 을 사용하여 얻은 리스트의 첫번째 요소

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [111]:
m = regex2.search(txt)
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [112]:
txt[m.start():m.end()]

'dave@google.com'

In [113]:
regex2.match(txt) 

# 문자열의 첫번째 부분과 끝 부분에 대해 일치하는지 판단
### 현재는 첫부분부터 일치하지 않기 때문에 None을 반환한
print(regex2.match(txt))


None


### 패턴에 해당하는 문자열 치환하기
#### regex.sub( 치환할 문자열, 문자열 )

In [114]:
print(txt)

Dave dave@google.com
Steve steve@gmail.com
Rob rob@gamil.com


In [115]:
print(regex2.sub('치환하고 붙일 요소',txt))
# txt 라는 문자열에서 regex2하는 패턴(여기서는 이메일 형식)에 해당하는 것을 '치환하고 붙일 요소'라는 텍스트로 치환

Dave 치환하고 붙일 요소
Steve 치환하고 붙일 요소
Rob 치환하고 붙일 요소


In [116]:
pattern2 = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
# 이렇게 나누고 싶은 경우 각각에 대해 괄호를 씌워준다.

In [117]:
regex3 = re.compile(pattern2, flags = re.IGNORECASE) # 패턴을 컴파일한 후

In [118]:
m2 = regex3.match('lhj950211@naver.com')
m2

<_sre.SRE_Match object; span=(0, 19), match='lhj950211@naver.com'>

In [119]:
m2.groups()

('lhj950211', 'naver', 'com')

In [120]:
regex3.split('lhj950211@naver.com')

['', 'lhj950211', 'naver', 'com', '']

In [141]:
regex3.findall(txt)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gamil', 'com'),
 ('lbrojae2', 'gmail', 'com'),
 ('jaebrother2', 'hanyang.ac', 'kr')]

#### 패턴으로 지정한 객체의 각각 요소에 접근하는 방법
#### ' ( ) ( ) ( ) .... '  각각 괄호 에 \1, \2, 등 \해당 번째 숫자 형식으로 접근한다.


In [142]:
print(regex3.sub(r'유저이름 : \1 , 도메인 : \2, suffix : \3 ', txt))
# txt 에서 regex3라는 패턴과 일치하는 것을 sub 메서드를 사용하여 치환한다.
# 이 때 치환되는 값들은 regex3 객체를 만들 때 패턴을 보면 되는데 ( )( )( ) 처럼 괄호 세개 형식으로
# 이 각각의 값들에 대해 접근하여 \1 , \2, \3 처럼 넣어준다

## 즉 결과 값은 sample@smple.com 과 같은 문자열이 위의 지정 문자열로 치환되면서 각각의 \1 값들은 매칭된 값으로 채워진다.

Dave 유저이름 : dave , 도메인 : google, suffix : com 
Steve 유저이름 : steve , 도메인 : gmail, suffix : com 
Rob 유저이름 : rob , 도메인 : gamil, suffix : com 
Hojae 유저이름 : lbrojae2 , 도메인 : gmail, suffix : com 
hozi 유저이름 : jaebrother2 , 도메인 : hanyang.ac, suffix : kr 


## 7.4.3 pandas 의 벡터화된 문자열 함수
### 데이터를 다듬고 정규화하는 작업
### Series 의 str 메서드 : pd.Series.str

In [143]:
txt

'Dave dave@google.com\nSteve steve@gmail.com\nRob rob@gamil.com\nHojae lbrojae2@gmail.com\nhozi jaebrother2@hanyang.ac.kr'